# Business Process Agent

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

## Setup

In [2]:
from helper import get_openai_api_key
from helper import parameterize_sparql

from langchain_openai import OpenAIEmbeddings, ChatOpenAI

In [3]:
openai_api_key = get_openai_api_key()

embedding_model = OpenAIEmbeddings(model="text-embedding-3-large")
llm = ChatOpenAI(
    model="gpt-4o",
    temperature=0,
    max_tokens=None,
    max_retries=2)

In [4]:
from rdflib import Dataset

graph = Dataset(default_union=True)
graph.parse("./ro_shared_data/odata_knowledge_graph.ttl",
            format="turtle")

<Graph identifier=urn:x-rdflib:default (<class 'rdflib.graph.Graph'>)>

## Fetch API specifications

In [5]:
from helper import fetch_entity_specification

In [6]:
fetch_entity_specification(graph,
    entity_set_uri="http://data.example.org/Service/API_PURCHASEORDER_2/EntitySet/PURCHASEORDER"
)

{'service': 'API_PURCHASEORDER_2',
 'entity_set': 'PURCHASEORDER',
 'properties': {'PurchaseOrder': {'type': 'String',
   'is_key': 'true',
   'max_length': 10,
   'fixed_values': []},
  'PurchaseOrderDate': {'type': 'Date',
   'is_key': 'None',
   'max_length': None,
   'fixed_values': []},
  'Supplier': {'type': 'String',
   'is_key': 'None',
   'max_length': 10,
   'fixed_values': []},
  'PurchasingOrganization': {'type': 'String',
   'is_key': 'None',
   'max_length': 4,
   'fixed_values': []},
  'PurchasingGroup': {'type': 'String',
   'is_key': 'None',
   'max_length': 3,
   'fixed_values': []},
  'PurchasingProcessingStatus': {'type': 'String',
   'is_key': 'None',
   'max_length': 2,
   'fixed_values': []},
  'PaymentTerms': {'type': 'String',
   'is_key': 'None',
   'max_length': 4,
   'fixed_values': []},
  'DocumentCurrency': {'type': 'String',
   'is_key': 'None',
   'max_length': 3,
   'fixed_values': []}},
 'navigations': [{'target_entity_set': 'PurchaseOrderItem',
   'ca

## Discovery Tool

In [7]:
import pickle

with open("../ro_shared_data/entity_sets_index.pickle", "rb") as file:
    index = pickle.load(file)

with open("../ro_shared_data/entity_set_uris.pickle", "rb") as file:
    entity_set_uris = pickle.load(file)

In [8]:
from helper import discover_apis_and_process

In [9]:
query = """Show me the active purchase orders in
purchasing group 002 and purchasing organization 3000"""
discover_apis_and_process(
    query=query,
    graph=graph,
    index=index,
    entity_set_uris=entity_set_uris,
    embedding_model=embedding_model,
)

{'entity_sets': {'http://data.example.org/Service/API_CNTRLREQFORQUOTATION/EntitySet/CNTRLREQFORQUOTATIONBIDDER',
  'http://data.example.org/Service/API_PLANNEDORDER/EntitySet/PLANNEDORDERHEADER',
  'http://data.example.org/Service/API_PURCHASEORDER_2/EntitySet/PURCHASEORDER',
  'http://data.example.org/Service/API_PURCHASEORDER_2/EntitySet/PURCHASEORDERITEM',
  'http://data.example.org/Service/API_PURCHASEREQUISITION_2/EntitySet/PURCHASEREQN',
  'http://data.example.org/Service/API_PURCHASEREQUISITION_2/EntitySet/PURCHASEREQNITEM'},
 'process_information': ['PurchaseOrder depends on PurchaseReqn']}

In [10]:
from langchain_core.tools import tool

@tool
def discover_apis(user_query: str) -> dict:
    """Discovers relevant API specifications and process information
    based on the user query.

    Args:
            user_query (str): user query

    Returns:
            discovery: (dict) of the form {'api_specs': api_specs,
            'process_information': process_information}
    """
    discovery = discover_apis_and_process(
        query=user_query,
        graph=graph,
        index=index,
        entity_set_uris=entity_set_uris,
        embedding_model=embedding_model,
    )
    api_specs = [
        fetch_entity_specification(graph ,entity_set_uri)
        for entity_set_uri in discovery["entity_sets"]
    ]
    return {
        "api_specs": api_specs,
        "process_information": discovery["process_information"],
    }

## Tools for calling mocked Data APIs

In [11]:
from helper import (data_prs, data_pos, data, 
                    post_data_mock, get_data_mock, 
                    display_pos, display_prs)

In [12]:
display_prs(data_prs)

,Purchase Requisition,Item,Material,Requested Quantity,Purchasing Group,Purchasing Organization
0,4000000001,00010,ruler,1,005,3000
1,4000000002,00010,pencil,5,002,3000
2,4000000002,00020,pen,3,002,3000


In [13]:
display_pos(data_pos)

,Purchase Order,Purchasing Group,Purchasing Organization,Processing Status,Item,Material,Order Quantity,Linked Requisition,Linked Requisition Item
0,4500000001,005,3000,01,00010,ruler,1,4000000001,00010
1,4500000002,002,3000,02,00010,pencil,5,4000000002,00010
2,4500000003,002,3000,02,00010,pen,3,4000000002,00020


In [14]:
post_data_mock(
    data=data,
    service_name="API_PURCHASEORDER_2",
    entity_set="PURCHASEORDER",
    payload={
        "PurchaseOrderItem": [
            {"Material": "mouse", "OrderQuantity": 5},
            {"Material": "keyboard", "OrderQuantity": 3},
        ]
    },
)
display_pos(data_pos)

Calling: POST /sap/opu/odata/sap/API_PURCHASEORDER_2/PURCHASEORDER with payload {'PurchaseOrderItem': [{'Material': 'mouse', 'OrderQuantity': 5}, {'Material': 'keyboard', 'OrderQuantity': 3}]}


,Purchase Order,Purchasing Group,Purchasing Organization,Processing Status,Item,Material,Order Quantity,Linked Requisition,Linked Requisition Item
0,4500000001,005,3000,01,00010,ruler,1,4000000001,00010
1,4500000002,002,3000,02,00010,pencil,5,4000000002,00010
2,4500000003,002,3000,02,00010,pen,3,4000000002,00020
3,4500000004,None,None,None,00010,mouse,5,None,None
4,4500000004,None,None,None,00020,keyboard,3,None,None


In [15]:
get_data_mock(
    data=data,
    service_name="API_PURCHASEORDER_2",
    entity_set="PURCHASEORDER",
    filter_string="""PurchasingGroup eq '005'
    and PurchasingOrganization eq '3000'""",
    selects_string="PurchaseOrder,PurchasingGroup,PurchasingOrganization",
)


Calling: GET /sap/opu/odata/sap/API_PURCHASEORDER_2/PURCHASEORDER?$filter=PurchasingGroup eq '005'
    and PurchasingOrganization eq '3000'&$select=PurchaseOrder,PurchasingGroup,PurchasingOrganization


[{'PurchaseOrder': '4500000001',
  'PurchasingGroup': '005',
  'PurchasingOrganization': '3000'}]

In [16]:
@tool
def get_api(service_name: str, entity_set: str,
    filter_string: str | None = None, selects_string: str | None = None,
) -> dict:
    """Fetches data from an API based on the provided parameters.

    Args:
            service_name (str): Name of the service.
            entity_set (str): Name of the entity set to query.
            filter_string (str, optional): Filter conditions for the query. Defaults to None.
            selects_string (str, optional): Fields to select in the response. Defaults to None.

    Returns:
            dict: The response from the API.
    """
    return get_data_mock(data=data, service_name=service_name,
        entity_set=entity_set, filter_string=filter_string, 
        selects_string=selects_string,
    )

@tool
def post_data_api(service_name: str, entity_set: str, 
                  payload: dict | None = None) -> dict:
    """Posts data to an API based on the provided parameters.

    Args:
            service_name (str): Name of the service.
            entity_set (str): Name of the entity set to post data to.
            payload (dict): The data to be posted.

    Returns:
            dict: The response from the API.
    """
    return post_data_mock(
        data=data, service_name=service_name, entity_set=entity_set, payload=payload
    )

## Define Agent

In [17]:
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.prompts import ChatPromptTemplate

In [18]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are a helpful assistant that can discover and
            call OData APIs to serve user requests.""",
        ),
        (
            "human",
            """
            1. Discover the relevant APIs and process information based
            on the user query. Use the full user query to discover the
            APIs.
            2. If you discover business process information relevant to
            the user query, make sure to follow the process to serve
            the user request.
            3. If an entityset has navigations, these can be used to
            create parent and child object in one API call to the parent.
            The payload may look like this:
            {{
                "parent_property_1": "value_1",
                "parent_property_2": "value_2",
                "child_entity_set_name": [
                {{"child_property_1": "value_3"}},
                {{"child_property_2": "value_4"}}
                ]
            }}

			- DO NOT create child objects without creating the parent 
              object first. 
            - Pay attention to the cardinality of the navigations. 
              If the cardinality is one_to_many, the dependent entity set
              can be a list of objects.
			- Do not invent values, for properties that are not specified,
              to not include them in the payload.        
		    - Use all fields that you're given values for 
              (either by the user or from previous tool calls) which match
              API properties in the payload.
            - If you create objects, do not include the key properties 
              in the payload. The keys properties are generated by 
              the backend.
			- When referring to previously created objects,
              use the keys of the objects in the payload 
			- If you create objects, do not query them to check.
            """,
        ),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

In [19]:
agent = create_tool_calling_agent(
    llm=llm, tools=[discover_apis, get_api, post_data_api], prompt=prompt
)

agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent,
    tools=[get_api, post_data_api, discover_apis],
    verbose=True,
    return_intermediate_steps=True,
)

## Example 1: Get active Purchase Orders

In [20]:
res = agent_executor.invoke(
    {
        "input": """Show me active purchase orders in
        purchasing group 002 and purchasing organization 3000"""
    }
)



> Entering new AgentExecutor chain...

Invoking: `discover_apis` with `{'user_query': 'Show me active purchase orders in purchasing group 002 and purchasing organization 3000'}`


{'api_specs': [{'service': 'API_PURCHASEREQUISITION_2', 'entity_set': 'PURCHASEREQNITEM', 'properties': {'RequestedQuantity': {'type': 'Decimal', 'is_key': 'None', 'max_length': None, 'fixed_values': []}, 'PurchaseRequisitionItem': {'type': 'String', 'is_key': 'true', 'max_length': 5, 'fixed_values': []}, 'PurchaseRequisition': {'type': 'String', 'is_key': 'true', 'max_length': 10, 'fixed_values': []}, 'Material': {'type': 'String', 'is_key': 'None', 'max_length': 40, 'fixed_values': []}, 'PurchasingOrganization': {'type': 'String', 'is_key': 'None', 'max_length': 4, 'fixed_values': []}, 'PurchasingGroup': {'type': 'String', 'is_key': 'None', 'max_length': 3, 'fixed_values': []}}, 'navigations': [{'target_entity_set': 'PurchaseReqn', 'cardinality': 'one_to_one'}]}, {'service': 'API_CNTRLREQFORQUOTATION', 'e

In [21]:
fixed_values_insert = """
PREFIX odata: <http://example.org/odata#>
INSERT DATA {
    <http://data.example.org/Service/API_PURCHASEORDER_2/EntityType/PURCHASEORDER_TYPE/Property/PURCHASINGPROCESSINGSTATUS> odata:valueHelp 
        [ odata:key "01" ; odata:value "In process" ],
        [ odata:key "02" ; odata:value "Active" ],
        [ odata:key "03" ; odata:value "In release" ],
        [ odata:key "04" ; odata:value "Partially released" ],
        [ odata:key "05" ; odata:value "Release completed" ],
        [ odata:key "08" ; odata:value "Rejected" ] .
}
"""
graph.update(fixed_values_insert)

In [22]:
res = agent_executor.invoke(
    {
        "input": """Show me the active purchase orders in
        purchasing group 002 and purchasing organization 3000"""
    }
)



> Entering new AgentExecutor chain...

Invoking: `discover_apis` with `{'user_query': 'Show me the active purchase orders in purchasing group 002 and purchasing organization 3000'}`


{'api_specs': [{'service': 'API_PURCHASEREQUISITION_2', 'entity_set': 'PURCHASEREQNITEM', 'properties': {'RequestedQuantity': {'type': 'Decimal', 'is_key': 'None', 'max_length': None, 'fixed_values': []}, 'PurchaseRequisitionItem': {'type': 'String', 'is_key': 'true', 'max_length': 5, 'fixed_values': []}, 'PurchaseRequisition': {'type': 'String', 'is_key': 'true', 'max_length': 10, 'fixed_values': []}, 'Material': {'type': 'String', 'is_key': 'None', 'max_length': 40, 'fixed_values': []}, 'PurchasingOrganization': {'type': 'String', 'is_key': 'None', 'max_length': 4, 'fixed_values': []}, 'PurchasingGroup': {'type': 'String', 'is_key': 'None', 'max_length': 3, 'fixed_values': []}}, 'navigations': [{'target_entity_set': 'PurchaseReqn', 'cardinality': 'one_to_one'}]}, {'service': 'API_CNTRLREQFORQUOTATION'

## Example 2: Create Purchase Order

In [23]:
res = agent_executor.invoke(
    {
        "input": """Create a purchase order for 5 pencils in
        purchasing group 002 and purchasing organization 3000"""
    }
)



> Entering new AgentExecutor chain...

Invoking: `discover_apis` with `{'user_query': 'Create a purchase order for 5 pencils in purchasing group 002 and purchasing organization 3000'}`


{'api_specs': [{'service': 'API_PURCHASEREQUISITION_2', 'entity_set': 'PURCHASEREQNITEM', 'properties': {'RequestedQuantity': {'type': 'Decimal', 'is_key': 'None', 'max_length': None, 'fixed_values': []}, 'PurchaseRequisitionItem': {'type': 'String', 'is_key': 'true', 'max_length': 5, 'fixed_values': []}, 'PurchaseRequisition': {'type': 'String', 'is_key': 'true', 'max_length': 10, 'fixed_values': []}, 'Material': {'type': 'String', 'is_key': 'None', 'max_length': 40, 'fixed_values': []}, 'PurchasingOrganization': {'type': 'String', 'is_key': 'None', 'max_length': 4, 'fixed_values': []}, 'PurchasingGroup': {'type': 'String', 'is_key': 'None', 'max_length': 3, 'fixed_values': []}}, 'navigations': [{'target_entity_set': 'PurchaseReqn', 'cardinality': 'one_to_one'}]}, {'service': 'API_CNTRLREQFORQUOTATI

In [24]:
display_prs(data_prs)

,Purchase Requisition,Item,Material,Requested Quantity,Purchasing Group,Purchasing Organization
0,4000000001,00010,ruler,1,005,3000
1,4000000002,00010,pencil,5,002,3000
2,4000000002,00020,pen,3,002,3000
3,4000000003,00010,pencil,5,002,3000


In [25]:
display_pos(data_pos)

,Purchase Order,Purchasing Group,Purchasing Organization,Processing Status,Item,Material,Order Quantity,Linked Requisition,Linked Requisition Item
0,4500000001,005,3000,01,00010,ruler,1,4000000001,00010
1,4500000002,002,3000,02,00010,pencil,5,4000000002,00010
2,4500000003,002,3000,02,00010,pen,3,4000000002,00020
3,4500000004,None,None,None,00010,mouse,5,None,None
4,4500000004,None,None,None,00020,keyboard,3,None,None
